In [67]:
! wget --no-check-certificate "https://t1.ipcamlive.com/timelapses/5aec8d9905b9a/2023-09-12/video.mp4"

--2023-09-20 12:21:04--  https://t1.ipcamlive.com/timelapses/5aec8d9905b9a/2023-09-12/video.mp4
Resolving t1.ipcamlive.com (t1.ipcamlive.com)... 173.198.196.110
Connecting to t1.ipcamlive.com (t1.ipcamlive.com)|173.198.196.110|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 36492417 (35M) [video/mp4]
Saving to: ‘video.mp4’

video.mp4           100%[===================>]  34,80M  13,8MB/s    in 2,5s    

2023-09-20 12:21:07 (13,8 MB/s) - ‘video.mp4’ saved [36492417/36492417]



In [76]:
import os
import wget
import subprocess

# Define the path to the directory containing satellite images
path_to_satellite_images = '/home/idisc02/Desktop/i2sc/detect-car-LR/data/data/Ocean_City2/7bafda78-d88c-49fa-aa63-b975b40bfec9/PSScene/'

# Iterate over the files in the directory
for filename in os.listdir(path_to_satellite_images):
    if filename.endswith('.tif'):
        # Extract date and time from the filename (assuming a specific format)
        parts = filename.split('_')
        if len(parts) >= 2:
            date = f"{parts[0][:4]}-{parts[0][4:6]}-{parts[0][6:]}"
            
            
            # Convert date and time to the desired format (e.g., "2023-06-25_15:30:14")
            video_url = f"https://t1.ipcamlive.com/timelapses/5aec8d9905b9a/{date}/video.mp4"
        
            # Define the output file path for the downloaded video
            output_video_file = os.path.join('/home/idisc02/Desktop/i2sc/detect-car-LR/Webcam_Frame_Extraction/Videos/', f"{date}.mp4")
            
            # Download the video using wget
            # wget.download(video_url, out=output_video_file, "--no-check-certificate")

            subprocess.call(['wget', '--no-check-certificate', '-O', output_video_file, video_url])


--2023-09-20 15:46:37--  https://t1.ipcamlive.com/timelapses/5aec8d9905b9a/2023-08-08/video.mp4
Resolving t1.ipcamlive.com (t1.ipcamlive.com)... 173.198.196.110
Connecting to t1.ipcamlive.com (t1.ipcamlive.com)|173.198.196.110|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 36524270 (35M) [video/mp4]
Saving to: ‘/home/idisc02/Desktop/i2sc/detect-car-LR/Webcam_Frame_Extraction/Videos/2023-08-08.mp4’

     0K .......... .......... .......... .......... ..........  0%  246K 2m25s
    50K .......... .......... .......... .......... ..........  0%  500K 1m48s
   100K .......... .......... .......... .......... ..........  0%  492K 96s
   150K .......... .......... .......... .......... ..........  0%  502K 89s
   200K .......... .......... .......... .......... ..........  0%  160M 71s
   250K .......... .......... .......... .......... ..........  0%  501K 71s
   300K .......... .......... .......... .......... ..

Frame Extraction from the TimeLapse Video (Ocean City Inlet Parking Lot)

In [30]:

import torch

from transformers import AutoImageProcessor, AutoModelForObjectDetection
#from transformers import pipeline

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import io
from random import choice

image_processor_tiny = AutoImageProcessor.from_pretrained("hustvl/yolos-tiny")
model_tiny = AutoModelForObjectDetection.from_pretrained("hustvl/yolos-tiny")

import gradio as gr


COLORS = ["#ff7f7f", "#ff7fbf", "#ff7fff", "#bf7fff",
            "#7f7fff", "#7fbfff", "#7fffff", "#7fffbf",
            "#7fff7f", "#bfff7f", "#ffff7f", "#ffbf7f"]

fdic = {
    "family" : "DejaVu Serif",
    "style" : "normal",
    "size" : 18,
    "color" : "yellow",
    "weight" : "bold"
}


def get_figure(in_pil_img, in_results):
    plt.figure(figsize=(16, 10))
    plt.imshow(in_pil_img)
    ax = plt.gca()

    for score, label, box in zip(in_results["scores"], in_results["labels"], in_results["boxes"]):
        selected_color = choice(COLORS)

        box_int = [i.item() for i in torch.round(box).to(torch.int32)]
        x, y, w, h = box_int[0], box_int[1], box_int[2]-box_int[0], box_int[3]-box_int[1]
        #x, y, w, h = torch.round(box[0]).item(), torch.round(box[1]).item(), torch.round(box[2]-box[0]).item(), torch.round(box[3]-box[1]).item()

        ax.add_patch(plt.Rectangle((x, y), w, h, fill=False, color=selected_color, linewidth=3, alpha=0.8))
        ax.text(x, y, f"{model_tiny.config.id2label[label.item()]}: {round(score.item()*100, 2)}%", fontdict=fdic, alpha=0.8)

    plt.axis("off")

    # return plt.gcf()
def infer(in_pil_img, in_model="yolos-tiny", in_threshold=0.9):
    target_sizes = torch.tensor([in_pil_img.size[::-1]])

    if in_model == "yolos-small":
        inputs = image_processor_tiny(images=in_pil_img, return_tensors="pt")
        outputs = model_tiny(**inputs)

            # convert outputs (bounding boxes and class logits) to COCO API
        results = image_processor_tiny.post_process_object_detection(outputs, threshold=in_threshold, target_sizes=target_sizes)[0]

    else:
        inputs = image_processor_tiny(images=in_pil_img, return_tensors="pt")
        outputs = model_tiny(**inputs)

            # convert outputs (bounding boxes and class logits) to COCO API
        results = image_processor_tiny.post_process_object_detection(outputs, threshold=in_threshold, target_sizes=target_sizes)[0]

        # figure = get_figure(in_pil_img, results)

        # buf = io.BytesIO()
        # figure.savefig(buf, bbox_inches='tight')
        # buf.seek(0)
        # output_pil_img = Image.open(buf)

        # return output_pil_img, results
    return results

def count_cars(img, threshold=0.6):
    Results = []
    for i in range(0,4):
        for j in range(0,4):
            img_arr = np.array(img)[180*i:180+180*i,320*j:320+320*j,:3]
            img_new = Image.fromarray(img_arr)
            results = infer(img_new, in_model="yolos-tiny", in_threshold=threshold)
            Results.append(results)
    count = []
    for in_results in Results:
        count.extend([i.item() for i in in_results['labels'] if i==3 or i == 8])

    return len(count)

import os
# Define the folder containing the images
image_folder = "/home/idisc02/Desktop/i2sc/detect-car-LR/Webcam_Frame_Extraction/Frames2"

# Define the threshold for car detection
threshold = 0.6

# Initialize a counter to keep track of the total number of cars
total_cars = 0

# Loop through all the image files in the folder
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Construct the full path to the image file
        image_path = os.path.join(image_folder, filename)

        # Open the image using PIL
        img = Image.open(image_path)

        # Count the cars in the image
        cars = count_cars(img, threshold=threshold)

        # Increment the total car count
        total_cars += cars

        # Optionally, print the car count for each image
        print(f"Number of cars in {filename}: {cars}")

        # # Display the image with car count
        # plt.imshow(img)
        # plt.title(f"Number of cars: {cars}")
        # plt.axis('off')
        # plt.show()



Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Number of cars in outputframe_2023-07-07_1530.jpg: 156
Number of cars in outputframe_2023-07-13_1457.jpg: 214
Number of cars in outputframe_2023-06-30_1530.jpg: 174
Number of cars in outputframe_2023-08-13_1456.jpg: 229
Number of cars in outputframe_2023-08-11_1458.jpg: 196
Number of cars in outputframe_2023-07-28_1458.jpg: 158
Number of cars in outputframe_2023-08-08_1530.jpg: 202
Number of cars in outputframe_2023-06-25_1530.jpg: 212
Number of cars in outputframe_2023-07-19_1457.jpg: 184
Number of cars in outputframe_2023-08-07_1529.jpg: 184
Number of cars in outputframe_2023-07-29_1457.jpg: 158
Number of cars in outputframe_2023-07-17_1454.jpg: 221
Number of cars in outputframe_2023-07-13_1450.jpg: 216
Number of cars in outputframe_2023-07-23_1457.jpg: 226
Number of cars in outputframe_2023-07-11_1452.jpg: 189
Number of cars in outputframe_2023-08-09_1533.jpg: 167
Number of cars in outputframe_2023-06-30_1454.jpg: 185
Number of cars in outputframe_2023-09-14_1457.jpg: 10
Number of c

In [34]:
import cv2
import os
import datetime

# Function to extract the frame closest to the target time
def extract_frame_by_time(video_file, target_time, output_file, video_start_time, video_end_time):
    cap = cv2.VideoCapture(video_file)

    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    # Convert the video start time to a datetime object
    video_start_time = datetime.datetime.strptime(video_start_time, "%H:%M")
    video_end_time = datetime.datetime.strptime(video_end_time, "%H:%M")

    # Calculate the target time with the offset time
    target_datetime = datetime.datetime.strptime(target_time, "%H:%M")

    # Calculate the total number of frames in the video
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

    # Calculate the video duration in seconds
    video_duration_seconds = (video_end_time - video_start_time).total_seconds()

    # Calculate the frame rate (frames per second)
    frame_rate = total_frames / video_duration_seconds

    # Calculate the target frame number based on the actual start time
    target_frame_number = int((target_datetime - video_start_time).total_seconds() * frame_rate)

    # Check if the target frame number is within the bounds of the video
    if target_frame_number < 0 or target_frame_number >= total_frames:
        print("No matching frame found within the video.")
        return

    cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame_number)

    ret, frame = cap.read()

    if ret:
        # Create the output folder if it doesn't exist
        os.makedirs(os.path.dirname(output_file), exist_ok=True)

        cv2.imwrite(output_file, frame)
        print(f"Frame extracted at {target_time}.")

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":

    video_folder = "/home/idisc02/Desktop/i2sc/detect-car-LR/Webcam_Frame_Extraction/Videos"  # Folder containing video files
    output_folder = "/home/idisc02/Desktop/i2sc/detect-car-LR/Webcam_Frame_Extraction/Frames3"  # Folder to save extracted frames
    path_to_satellite_images = "/home/idisc02/Desktop/i2sc/detect-car-LR/data/data/Ocean_City2/7bafda78-d88c-49fa-aa63-b975b40bfec9/PSScene"

    # Get a list of .tif files in the satellite images folder
    satellite_image_files = [f for f in os.listdir(path_to_satellite_images) if f.endswith(".tif")]

    for satellite_image_file in satellite_image_files:
        # Extract the target time from the satellite image file name
        parts = satellite_image_file.split("_")  # Assuming the target time is in the second part of the filename
        date = f"{parts[0][:4]}-{parts[0][4:6]}-{parts[0][6:]}"
        target_time = f"{parts[1][:2]}:{parts[1][2:4]}"
        # print(target_time)

        video_file = os.path.join(video_folder, f"{date}.mp4")

        # Define the video start and end times (modify as needed)
        video_start_time = "00:11"  # Replace with the video start time in 24-hour format
        video_end_time = "23:59"  # Replace with the video end time in 24-hour format

        # Create the output file name using string formatting
        video_base_name = os.path.splitext(os.path.basename(video_file))[0]
        # output_file = os.path.join(output_folder, f"outputframe_{video_base_name}_{target_time.replace(':', '')}.jpg")
        output_file = os.path.join(output_folder, f"{date.replace('-', '')}_{target_time.replace(':', '')}_frame.jpg")

        # Call the extract_frame_by_time function
        extract_frame_by_time(video_file, target_time, output_file, video_start_time, video_end_time)

        

Frame extracted at 15:30.
Frame extracted at 14:52.
Frame extracted at 14:54.
Frame extracted at 14:57.
Frame extracted at 15:33.
Frame extracted at 15:32.
Frame extracted at 14:48.
Frame extracted at 15:29.
Frame extracted at 15:35.
Frame extracted at 14:58.
Frame extracted at 15:34.
Frame extracted at 14:57.
Frame extracted at 14:48.
Frame extracted at 14:57.
Frame extracted at 14:47.
Frame extracted at 14:58.
Frame extracted at 15:30.
Frame extracted at 14:58.
Frame extracted at 14:56.
Frame extracted at 14:50.
Frame extracted at 15:37.
Frame extracted at 14:52.
Frame extracted at 15:30.
Frame extracted at 14:50.
Frame extracted at 14:57.
Frame extracted at 14:53.
Frame extracted at 15:31.
Frame extracted at 14:52.
Frame extracted at 14:55.
Frame extracted at 15:32.
Frame extracted at 15:32.
Frame extracted at 14:54.
Frame extracted at 15:30.
Frame extracted at 14:54.
Frame extracted at 14:47.
Frame extracted at 14:54.
Frame extracted at 14:57.
Frame extracted at 14:53.
Frame extrac

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55a49a819880] moov atom not found


In [41]:
import pandas as pd
import os
from PIL import Image

# Define the folder containing the satellite images
path_to_satellite_images = '/home/idisc02/Desktop/i2sc/detect-car-LR/data/data/Ocean_City2/7bafda78-d88c-49fa-aa63-b975b40bfec9/PSScene/'

# Define the folder containing the extracted frames
path_to_extracted_frames = '/home/idisc02/Desktop/i2sc/detect-car-LR/Webcam_Frame_Extraction/Frames3/'

# Define the threshold for car detection
threshold = 0.6

# Initialize a list to store the results
data = []

# Get a list of satellite image filenames
satellite_image_filenames = os.listdir(path_to_satellite_images)
frame_filenames = os.listdir(path_to_extracted_frames)

# Loop through all the satellite image filenames
for satellite_image_filename in satellite_image_filenames:
    if satellite_image_filename.endswith('.tif'):
        # Extract the frame filename from the satellite image filename
        frame_filename = None
        for filename in frame_filenames:
            if filename.startswith(satellite_image_filename[:13]):
                frame_filename = filename
                break

        if frame_filename is not None:
            # Construct the full path to the extracted frame file
            frame_file_path = os.path.join(path_to_extracted_frames, frame_filename)

            # Open the extracted frame file using PIL
            img = Image.open(frame_file_path)

            # Count the cars in the extracted frame
            cars = count_cars(img, threshold=threshold)

            # Add the results to the list
            data.append([satellite_image_filename, frame_filename, cars])

# Create a Pandas DataFrame from the list
df = pd.DataFrame(data, columns=["satellite_image_filename", "frame_name", "car_count"])

# Save the DataFrame to a CSV file
df.to_csv("/home/idisc02/Desktop/i2sc/detect-car-LR/Webcam_Frame_Extraction/car_count_results.csv", index=False)
